In [11]:
import json
import minsearch
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [12]:
# Load API key
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

# Initialize Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Load and prepare documents
documents_path = "documents.json" 
with open(documents_path, 'r', encoding='utf-8') as file:
    documents_raw = json.load(file)

documents = []

for course in documents_raw:  # Change from 'documents' to 'documents_raw'
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Refit the index with the expanded document set
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [13]:
def search(query, filter_course=None):
    boost = {'question': 3.0, 'section': 0.5}
    
    filter_dict = {}
    if filter_course:
        filter_dict = {'course': filter_course}
        
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=5
    )
    
    return results

In [14]:
def build_prompt(query, diagnosis_result, search_results):
    prompt_template = """
You are a dermatology AI assistant for the Acne Sense app. Your role is to provide helpful, accurate information about acne based on the user's skin analysis and questions.

SKIN ANALYSIS RESULT:
{diagnosis}

USER QUESTION: {question}

CONTEXT FROM KNOWLEDGE BASE:
{context}

Provide a helpful, compassionate response that:
1. Acknowledges their specific acne condition
2. Offers relevant treatment suggestions and skincare advice based on their acne type
3. Clearly indicates when professional dermatologist consultation is recommended
4. Uses only information from the provided context
5. Avoids making definitive medical diagnoses or prescribing specific medications
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(diagnosis=diagnosis_result, question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": 0.2,  # Lower temperature for more factual responses
            "max_output_tokens": 800,  # Appropriate length for detailed answers
            "top_p": 0.95,
            "top_k": 40,
        }
    )
    
    return response.text

In [15]:

def rag(query, diagnosis_result=None, filter_course=None):
    search_results = search(query, filter_course)
    prompt = build_prompt(query, diagnosis_result, search_results)
    answer = llm(prompt)
    return answer

In [16]:
# Test the basic LLM function
response = llm('write that this is a test')
print(response)

This is a test.



In [17]:
# Test with a question about the app
app_query = "How does the Acne Sense app protect my privacy?"
app_answer = rag(app_query, filter_course="acne-sense-app-info")
print("Question about the app:")
print(f"Q: {app_query}")
print(f"A: {app_answer}")
print("\n" + "="*50 + "\n")

Question about the app:
Q: How does the Acne Sense app protect my privacy?
A: Hello!  Your Acne Sense skin analysis currently shows no acne. That's great news!

Since your analysis shows no acne at this time, there's no need for specific treatment suggestions or skincare advice tailored to your acne type.  However, maintaining good skin health is always important.  To track your skin's condition over time and catch any potential future issues early, you can use the Acne Sense 'Progress' feature. Remember to take photos consistently, ideally every 2-4 weeks, under similar lighting conditions.

To learn more about how Acne Sense works and how to take photos for analysis, please refer to the app's help section.  If you notice any changes in your skin or develop acne in the future, the app can help you track your progress and assess your condition.

While Acne Sense can be a helpful tool for managing your skin, it's not a substitute for professional medical advice.  If you have any concern

In [18]:
# Test with a question about acne treatment
acne_query = "What should I do about papules on my face?"
acne_diagnosis = """
Acne Type: Mild inflammatory acne
Main Lesions Detected:
- Several papules (6)
- Few pustules (2)
Location: Primarily on forehead and cheeks
"""
acne_answer = rag(acne_query, acne_diagnosis, filter_course="acne-sense-knowledge-base")
print("Question about acne treatment:")
print(f"Q: {acne_query}")
print(f"A: {acne_answer}")
print("\n" + "="*50 + "\n")

Question about acne treatment:
Q: What should I do about papules on my face?
A: Hi there!  I understand you're dealing with mild inflammatory acne, specifically some papules on your forehead and cheeks.  That's completely manageable.

Based on your skin analysis showing several papules,  you can try over-the-counter treatments containing benzoyl peroxide or salicylic acid. These are commonly used to treat papules, which are small, raised pimples. Remember to avoid picking or squeezing them, as this can worsen inflammation and potentially lead to scarring.  

To introduce these new products, start with a low concentration and apply them only once a day, gradually increasing frequency as tolerated.  Remember to also incorporate a gentle cleanser and a non-comedogenic moisturizer into your routine.  Don't forget sunscreen during the day!

It's important to be patient and consistent with your skincare routine. If you don't see improvement after 2-3 months of consistently using over-the-cou

In [19]:
mixed_query = "How can I track my progress using benzoyl peroxide with the app?"
mixed_diagnosis = """
Acne Type: Mild comedonal acne with inflammatory elements
Main Lesions Detected:
- Several blackheads and whiteheads
- Few papules (3)
Location: Primarily on forehead and nose
"""
# No filter - let it search across all knowledge bases
mixed_answer = rag(mixed_query, mixed_diagnosis) 
print("Mixed question about treatment tracking in app:")
print(f"Q: {mixed_query}")
print(f"A: {mixed_answer}")

Mixed question about treatment tracking in app:
Q: How can I track my progress using benzoyl peroxide with the app?
A: Hi there!  Based on your Acne Sense analysis, you have mild comedonal acne with some inflammatory elements, mainly blackheads, whiteheads, and a few papules on your forehead and nose.  That's good news, as mild acne is often treatable with over-the-counter options.

You're using benzoyl peroxide, which is excellent!  Remember to apply it as directed, starting with once daily and increasing to twice daily only if your skin tolerates it well.  Benzoyl peroxide kills acne-causing bacteria and helps remove excess oil.  Be mindful that it can bleach fabrics, so protect your clothes and bedding.  You might experience some dryness, redness, or irritation – if this becomes severe, reduce the frequency of application.

To track your progress with benzoyl peroxide, use the Acne Sense 'Progress' feature. Take new analysis photos every 2-4 weeks, ensuring consistent lighting and a